In [3]:
# Import necessary libraries
import duckdb
import pandas as pd
import requests
from io import StringIO

# Step 1: Download the CSV data from S3
s3_url = "https://s3-us-west-1.amazonaws.com/epicactuaryhealth/patient_churn.csv"
response = requests.get(s3_url)

# Check if the request was successful
if response.status_code != 200:
    raise Exception(f"Failed to download data, status code: {response.status_code}")

# Read the CSV data into a pandas DataFrame
data = pd.read_csv(StringIO(response.text))

# Step 2: Connect to DuckDB (in-memory database)
conn = duckdb.connect(database=':memory:')

# Step 3: Load the DataFrame into DuckDB
conn.execute("CREATE TABLE patient_churn AS SELECT * FROM data")

# Step 4: Define and execute the SQL query to find the oldest beneficiary and mode age of living beneficiaries
query = """
WITH age_calculations AS (
    SELECT
        ID,
        (2015 - "Birth Year") AS age,
        "Death Year"
    FROM
        patient_churn
),

oldest_beneficiary AS (
    SELECT
        ID,
        age
    FROM
        age_calculations
    ORDER BY
        age DESC
    LIMIT 1
),

mode_age_living AS (
    SELECT
        age,
        COUNT(*) AS frequency
    FROM
        age_calculations
    WHERE
        "Death Year" IS NULL
    GROUP BY
        age
    ORDER BY
        frequency DESC
    LIMIT 1
)

SELECT
    ob.ID AS oldest_beneficiary_id,
    ob.age AS oldest_age,
    ma.age AS mode_age_living,
    ma.frequency AS mode_frequency
FROM
    oldest_beneficiary ob,
    mode_age_living ma;
"""

# Execute the query and fetch the results
result = conn.execute(query).fetchdf()

# Display the results
print(result)


   oldest_beneficiary_id  oldest_age  mode_age_living  mode_frequency
0                 124410         107               68            6019


In [4]:
# Import necessary libraries
import duckdb
import pandas as pd
import requests
from io import StringIO

# Step 1: Download the CSV data from S3
s3_url = "https://s3-us-west-1.amazonaws.com/epicactuaryhealth/patient_churn.csv"
response = requests.get(s3_url)

# Check if the request was successful
if response.status_code != 200:
    raise Exception(f"Failed to download data, status code: {response.status_code}")

# Read the CSV data into a pandas DataFrame
data = pd.read_csv(StringIO(response.text))

# Step 2: Connect to DuckDB (in-memory database)
conn = duckdb.connect(database=':memory:')

# Step 3: Load the DataFrame into DuckDB
conn.execute("CREATE TABLE patient_churn AS SELECT * FROM data")

# Step 4: Define and execute the SQL query to count deceased and total beneficiaries
query = """
WITH total_beneficiaries AS (
    SELECT COUNT(*) AS total_count
    FROM patient_churn
),
deceased_beneficiaries AS (
    SELECT COUNT(*) AS deceased_count
    FROM patient_churn
    WHERE "Death Year" IS NOT NULL
)
SELECT
    tb.total_count,
    db.deceased_count,
    ROUND((db.deceased_count::DECIMAL / tb.total_count) * 100, 2) AS deceased_proportion_percentage
FROM
    total_beneficiaries tb,
    deceased_beneficiaries db;
"""

# Execute the query and fetch the results
result = conn.execute(query).fetchdf()

# Display the results
print(result)


   total_count  deceased_count  deceased_proportion_percentage
0       124462           22612                           18.17
